In [1]:
from pyspark.sql import SparkSession
from pathlib import Path

In [2]:
spark = SparkSession.builder \
            .config("spark.driver.memory", "6g") \
            .config("spark.sql.adaptive.enabled", "True")\
            .master("local").getOrCreate();

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/26 15:30:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
root_fp = Path("./WK10_PARQUET")

### Category

In [4]:
cat_df = spark.read.parquet(str(root_fp / 'category'), inferSchema=True, header=True)
cat_df.createOrReplaceTempView("cat")
# spark.sql("""SELECT * FROM cat LIMIT 10""").show()

## chain

In [5]:
# chain_df = spark.read.parquet('./chain', inferSchema=True, header=True)
# chain_df.createOrReplaceTempView("chain")
# spark.sql("""SELECT * FROM chain LIMIT 10""").show()

## dma

In [6]:
dma_raw_df = spark.read.parquet(str(root_fp / './dma'), inferSchema=True, header=True)
dma_raw_df.createOrReplaceTempView("dma_raw")
# spark.sql("""SELECT * FROM dma_raw LIMIT 10""").show()
dma_df = spark.sql("""SELECT * FROM dma_raw WHERE dma_name='San Francisco-Oakland-San Jose, CA';""")

## Metric

In [7]:
met_raw_df = spark.read.parquet(str(root_fp / './metric'), inferSchema=True, header=True)
met_raw_df.createOrReplaceTempView("met_raw")
# spark.sql("""SELECT * FROM met_raw LIMIT 1""")
met_df = spark.sql("""SELECT * FROM met_raw WHERE category='cross_shopping' and sub_category = 'weekly'""")

## Place

In [8]:
plc_raw_df = spark.read.parquet(str(root_fp / './place'), inferSchema=True, header=True)
plc_raw_df.createOrReplaceTempView("plc_raw")
# spark.sql("""SELECT * FROM plc_raw LIMIT 1""")
plc_df = spark.sql("""SELECT * FROM plc_raw WHERE names LIKE '%Whole Food%';""")

## Join

In [9]:
dma_df = dma_df.withColumnRenamed("dma_code","dma_id")
j_df = plc_df.join(dma_df, on=["dma_id"]).join(met_df,["spatial_id"])
j_df.createOrReplaceTempView("j_df")

22/07/26 15:30:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
j_df.columns

['spatial_id',
 'dma_id',
 'place_id',
 'chain_id',
 'category_ids',
 'latitude',
 'longitude',
 'names',
 'address',
 'city',
 'state',
 'zip_code',
 'census_block_group',
 'cbsa_id',
 'local_timezone',
 'opening_date',
 'closing_date',
 'is_open',
 'is_high_confidence',
 'is_multiuse',
 'closed_hours',
 'ceiling_score',
 'dt',
 'dma_name',
 'dt',
 'category',
 'sub_category',
 'metric',
 'value',
 'value_units',
 'start_date',
 'end_date',
 'dt']

In [11]:
m_df = spark.sql("""SELECT metric, SUM(value) FROM j_df GROUP BY metric SORT BY metric DESC""")

In [12]:
m_df.count()

418

In [13]:
m_df.coalesce(1).write.csv('PIQ_EVAL_1_WEEK.csv', mode='overwrite')

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 5]
Traceback (most recent call last):
  File "/opt/venv/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/venv/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 